In [1]:
from utils import ReadTree
import pandas as pd
import numpy as np

In [2]:
treeData, treeConnectivity = ReadTree('Patient1/sim_0/stage.cco.root')
print(treeData[0])
Ids = [v[0] for v in treeData]

Reading Patient1/sim_0/stage.cco.root
	Number of vessels in the tree: 215 

[0, 40.0, 10.0, 3750.0, -6000.0, 0.0, 0.0, -2250.0, 0.0, 0.0]


reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName('stage_0.vtp')
reader.Update()
polydata = reader.GetOutput()
diameter = 2.0*1e4*VN.vtk_to_numpy(polydata.GetCellData().GetArray('radius'))
length   = 1e4*VN.vtk_to_numpy(polydata.GetCellData().GetArray('length'))
flow     = VN.vtk_to_numpy(polydata.GetCellData().GetArray('flow'))

x = np.zeros(polydata.GetNumberOfPoints())
y = np.zeros(polydata.GetNumberOfPoints())
z = np.zeros(polydata.GetNumberOfPoints())
for i in range(polydata.GetNumberOfPoints()):
        x[i],y[i],z[i] = polydata.GetPoint(i)


In [3]:
dflist = [[v[0], 2*v[1], v[3], v[2], v[4], v[5], v[6], v[7], v[8],v[9], v[0], v[0]+1] for v in treeData]
df = pd.DataFrame(dflist, columns=['Id', 'Diameter', 'Length', 'Flow', 'xprox','yprox','zprox',
                                   'xdist', 'ydist','zdist', 'From', 'To'])
df.head(-4)

,Id,Diameter,Length,Flow,xprox,yprox,zprox,xdist,ydist,zdist,From,To
0,0,80.0,3750.000000,10.0,-6000.0,0.0,0.0,-2250.0,0.0,0.0,0,1
1,1,80.0,390.000000,0.0,-2250.0,0.0,0.0,-2250.0,390.0,0.0,1,2
2,2,80.0,390.000000,0.0,-2250.0,390.0,0.0,-2250.0,780.0,0.0,2,3
3,3,80.0,390.000000,0.0,-2250.0,780.0,0.0,-2250.0,1170.0,0.0,3,4
4,4,80.0,390.000000,0.0,-2250.0,1170.0,0.0,-2250.0,1560.0,0.0,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
206,206,80.0,117.046999,0.0,-1270.0,-600.0,0.0,-1230.0,-490.0,0.0,206,207
207,207,80.0,86.023253,0.0,-1230.0,-490.0,0.0,-1160.0,-440.0,0.0,207,208
208,208,80.0,78.102497,0.0,-1160.0,-440.0,0.0,-1100.0,-390.0,0.0,208,209
209,209,80.0,80.000000,0.0,-1100.0,-390.0,0.0,-1020.0,-390.0,0.0,209,210


In [4]:
nodesProx = {v[0]:v[4:7] for v in treeData}
nodesDist = {v[0]+1:v[7:] for v in treeData}

nodesDist[0] = nodesProx[0]
for v in treeConnectivity[1:]:
    IdDistNode, IdProxNode = v[0]+1, v[1]+1
    df.at[v[0], 'From'] = IdProxNode
    df.at[v[0], 'To']   = IdDistNode

In [5]:
f = open('Network.dat', 'w')
f.write('Network for patient1, sim0, stage0\n')
f.write('8500 4000 20\t box dimensions in microns\n')
f.write('250 250 1\t number of tissue points in x,y,z directions\n')
f.write('10000\t outer bound distance\n')
f.write(f'{df.Length.max()}\t max. segment length.  (5. in test case)\n')
f.write('8  \t nodsegm, max. allowed number of segments per node\n')
f.write(f'{int(df.Id.max())}\t total number of segments\n')
f.write('name\t type\t from\t to\t diam\t\t flow\t\t HD\n')

39

In [6]:
for index, row in df.iterrows():
    f.write(f'{index}\t 5\t {int(row.From)}\t {int(row.To)}\t {row.Diameter}\t\t {row.Flow}\t 0.4\n')

In [7]:
f.write('name\t x\t y\t z\n')
for i in range(len(nodesDist)):
    x,y,z = nodesDist[i]
    f.write(f'{i}\t {x}\t {y}\t {z}\n')

f.write('1 total number of boundary nodes\n')
f.write('node\t bctyp\t press/flow\t HD\t PO2\t solute\n')
f.write(f'0\t 0\t {df.Flow.max()}\t 0.4\t 40.0\t 1.\n')
        

26

In [8]:
f.close()